After getting all the functions together and running

In [3]:
import subprocess
from nb_tools import file_transaction, file_exists
from orf_setup import prodigal
import os
import os.path as op
import glob
import pandas as pd

In [4]:
wd = "/mnt/scgc/simon/simonsproject/bats248_vs"

In [4]:
os.mkdir(op.join(wd, "prodigal/")

In [5]:
exesag = "AG-903-I06"
name = exesag
plate = "-".join(exesag.split("-")[:2])
contigfa = "/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/{plate}/{sag}_contigs.fasta".format(plate=plate, sag=exesag)
output = "/mnt/scgc/simon/simonsproject/bats248_vs"

In [6]:
outfiles = [os.path.join(output, "prodigal", name + "_proteins.fasta"),
                os.path.join(output, "prodigal", name + "_genes.fasta"),
                os.path.join(output, "prodigal", name + ".gbk"),
                os.path.join(output, "prodigal", name + ".scores")]

In [19]:
%%time
prodigal(contigfa, outfiles, verbose=True)

Running Prodigal on /mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/AG-903/AG-903-I06_contigs.fasta


CPU times: user 6 ms, sys: 21 ms, total: 27 ms
Wall time: 835 ms


['/mnt/scgc/simon/simonsproject/bats248_vs/prodigal/AG-903-I06_proteins.fasta',
 '/mnt/scgc/simon/simonsproject/bats248_vs/prodigal/AG-903-I06_genes.fasta',
 '/mnt/scgc/simon/simonsproject/bats248_vs/prodigal/AG-903-I06.gbk',
 '/mnt/scgc/simon/simonsproject/bats248_vs/prodigal/AG-903-I06.scores']

In [3]:
contigs = glob.glob("/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/*/*_contigs.fasta") + glob.glob("/mnt/scgc/simon/simonsproject/bats248_contigs/*/*_contigs.fasta")

In [ ]:
# loop to run prodigal, i was lazy and just let this run overnight in my interactive session
# should've submitted a job array instead

out_proteins = []
err_prots = []


for c in contigs:
    #break inserted so that i don't accidentally rerun this.
    break
    exesag = op.basename(c).split("_")[0]
    name = exesag
    plate = "-".join(exesag.split("-")[:2])
    output = "/mnt/scgc/simon/simonsproject/bats248_vs"
    outfiles = [os.path.join(output, "prodigal", name + "_proteins.fasta"),
                os.path.join(output, "prodigal", name + "_genes.fasta"),
                os.path.join(output, "prodigal", name + ".gbk"),
                os.path.join(output, "prodigal", name + ".scores")]
    try:
        p_proteins, p_genes, p_genbank, p_score = prodigal(c, outfiles, verbose=True)
    except Exception as inst:
        print(inst)
        err_prots.append(exesag)
        continue
    out_proteins.append(p_proteins)

In [8]:
len(out_proteins)

6397

All ORFs into one file next:

In [10]:
# !cat /mnt/scgc/simon/simonsproject/bats248_vs/prodigal/*proteins.fasta > /mnt/scgc/simon/simonsproject/bats248_vs/batch_prots.fasta
!cat /mnt/scgc/simon/simonsproject/bats248_vs/batch_prots.fasta /mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.fasta > /mnt/scgc/simon/simonsproject/bats248_vs/clustering/cdhit_in.fasta

In [12]:
in_fasta = '/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cdhit_in.fasta'
out_fasta = '/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cdhit_out.fasta'

In [52]:
cmd = "cd-hit -i {in_fa} -p 1 -c 0.90 -o {out_fa} -T 50 -M 150000 -d 0".format(in_fa = in_fasta, out_fa = out_fasta)
outlog = "/home/julia/out/180125_cdhit.out"

print("echo '{cmd}' | qsub -N cdhit -q scgc-route -V -l walltime=10:00:00,ncpus=50,mem=150G -j oe -o {log}".format(cmd = cmd, log=outlog))


echo 'cd-hit -i /mnt/scgc/simon/simonsproject/bats248_vs/clustering/cdhit_in.fasta -p 1 -c 0.90 -o /mnt/scgc/simon/simonsproject/bats248_vs/clustering/cdhit_out.fasta -T 50 -M 150000 -d 0' | qsub -N cdhit -q scgc-route -V -l walltime=10:00:00,ncpus=50,mem=150G -j oe -o /home/julia/out/180125_cdhit.out


ID ORFs that already fall into characterized clusters, and others that will need to be analyzed

In [68]:
from nb_tools import readfa
from collections import defaultdict
import itertools

In [75]:
seed_fa = '/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.fasta'
seeds = set()

for name, seq in readfa(open(seed_fa)):
    seeds.add(name.split()[0])
    
clstr = '/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cdhit_out.fasta.clstr'
cluster_map = defaultdict(list)
new_seeds = []
member_seeds = 0
old_seeds = 0

with open(clstr) as fh:
    for cluster_start, group in itertools.groupby(fh, lambda l: l[0] == '>'):
        members = []
        rep_seq = ''
        if not cluster_start: 
            for line in group:
                if "*" in line: 
                    rep_seq = line.split(",")[1].split("...")[0].replace(">",'').replace(" ","")
                
                members.append(line.split(",")[1].split("...")[0].replace(">",'').replace(" ",""))
        
        if len(rep_seq) == 0:
            continue
        
        seed_seq = None
        
        if rep_seq in seeds:
            seed_seq = rep_seq
            members.remove(rep_seq)
            old_seeds += 1
        else: 
            for m in members:
                if m in seeds:
                    seed_seq = m
                    member_seeds += 1
                    members.remove(m)
                    break
        
        if seed_seq is None:
            seed_seq = rep_seq
            members.remove(rep_seq)
            new_seeds.append(rep_seq)
        
        cluster_map[seed_seq] = members

In [76]:
print('there are', len(new_seeds), 'new seeds')
print('there are', old_seeds, 'old seeds')
print('there are', member_seeds, 'old seeds included in sequence clusters that will serve as the cluster seed')

there are 54393 new seeds
there are 70018 old seeds
there are 742502 old seeds included in sequence clusters that will serve as the cluster seed


In [77]:
len(cluster_map)

866913

Write out cluster map to file for future reference:

In [79]:
out_map = "/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cluster_map.txt"

def write_cluster_map(cmap, out_map):
    count = 0
    with open(out_map, "w") as oh:
        for k in cluster_map:
            print(k, k, sep = "\t", file=oh)
            for val in cluster_map[k]:
                print(val, k, sep="\t", file=oh)
                
def read_cluster_map(out_map):
    cmap = {}
    with open("/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cluster_map.txt") as ih:
        for l in ih:
            vec = l.strip().split("\t")
            cmap[vec[0]] = vec[1]
    return cmap

Write new seeds to a new fasta:

In [63]:
new_seed_fa = "/mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seeds.fasta"

with open(new_seed_fa, "w") as oh:
    for name, seq in readfa(open(clstr.replace(".clstr",''))):
        if name.split()[0] in new_seeds:
            print(">{}".format(name), file=oh)
            for i in range(0, len(seq), 60):
                print(seq[i:i+60], file=oh)

ID Similarity Groups for clustering to make mica run faster.

In [66]:
out70_fasta = new_seed_fa.replace(".fasta","_c70.fasta")
out70_fasta

'/mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seeds_c70.fasta'

In [67]:
def print_cdhit_sub(in_fasta, out_fasta, outlog, similarity=0.9):

    cmd = "cd-hit -i {in_fa} -p 1 -c {similarity} -o {out_fa} -T 50 -M 150000 -d 0".format(in_fa = in_fasta, out_fa = out_fasta, similarity=similarity)
    print("echo '{cmd}' | qsub -N cdhit -q scgc-route -V -l walltime=10:00:00,ncpus=50,mem=150G -j oe -o {log}".format(cmd = cmd, log=outlog))


In [68]:
print_cdhit_sub(new_seed_fa, out70_fasta, '180126_cdhit70.out', similarity=0.7)

echo 'cd-hit -i /mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seeds.fasta -p 1 -c 0.7 -o /mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seeds_c70.fasta -T 50 -M 150000 -d 0' | qsub -N cdhit -q scgc-route -V -l walltime=10:00:00,ncpus=50,mem=150G -j oe -o 180126_cdhit70.out


In [73]:
from phage_count_table import map_clstr_raw

In [78]:
clstr = "{}.clstr".format(out70_fasta)

cluster_map = map_clstr_raw(clstr, singles=True)

In [70]:
try:
    os.mkdir("/mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seed_groups/")
except:
    print("oh well")

In [80]:
from pyfaidx import Fasta
from nb_tools import write_fa_record

fa = Fasta(new_seed_fa)
count = 0
number = 0
missing = []

for k in cluster_map:
    seqs = [k] + cluster_map[k]
    count += len(seqs)
    with open("/mnt/scgc/simon/simonsproject/bats248_vs/clustering/new_seed_groups/newseeds_{}.fasta".format(number), "a") as oh:
        for s in seqs:
            try:
                rec = fa[s]
            except:
                missing.append(s)
                continue
                
            write_fa_record(rec.long_name, str(rec), oh)          

    if count > 1000:
        count = 0
        number += 1
print(number, "files created")

54 files created


In [81]:
try:
    os.mkdir(op.join(wd, "blastout","new_seeds"))
except:
    "oh well"

In [1]:
# Mica array submitted to pbs here: /mnt/stepanauskas_nfs/julia/pbs_subs/180126_newseeds_mica.sh
!cat /mnt/stepanauskas_nfs/julia/pbs_subs/180126_newseeds_mica.sh

#!/bin/bash                                                                                                             

## set name of PBS job                                                                                                  
#PBS -N newseeds_mica

## set the job array variable                                                                                           
#PBS -J 1-55                                                                                               

## send the environment variables with job                                                                              
#PBS -V

## set the queue                                                                                                        
#PBS -q scgc-route
#PBS -l walltime=24:00:00                                                                                               

#PBS -l ncpus=20                                                                           
                    

Job array ran overnight.  Wooo.  Now construct new phage table:

In [2]:
from nb_tools import id_virus_orfs, readfa, swap_cluster_map

In [9]:
micas = glob.glob(op.join(wd, "blastout","new_seeds",'*'))

In [10]:
new_micas

tbls = []

for m in micas:
    tbls.append(id_virus_orfs(m))
    
cluster_calls = pd.concat(tbls)

In [37]:
cluster_calls.to_csv(op.join(wd, "phage_hits_newseeds.csv"), index=False)

Import the cluster map I made earlier:

In [124]:
cmap = {}
with open("/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cluster_map.txt") as ih:
    for l in ih:
        vec = l.strip().split("\t")
        cmap[vec[0]] = vec[1]

In [81]:
len(cmap)

5323974

import phage hits tables

In [38]:
phage_hits = pd.concat([pd.read_csv(i) for i in glob.glob(op.join(wd, "phage_hits*"))])

In [44]:
from collections import Counter
Counter(phage_hits['p1'])

Counter({0: 732303, 1: 23851})

In [ ]:
from phage_count_table import phage_contig_table
from recruitment_for_vs import summarize_by_contig
from summary import virus_class

In [45]:
to_start =  '''AG−891−A17
AG−892−P18
AG−893−J23
AG−894−C07
AG−895−P08
AG−897−A15  
AG−903−F19
AG−903−I06
AG−904−O13
AG−907−C19
AG−907−I10
AG−908−F15
AG-909-A05
AG−910−E05
AG−912−O18
AG−913−C05
AG−913−C17'''.replace("−","-").split()

In [128]:
sag = to_start[0]

for sag in to_start:
    plate = "-".join(sag.split("-")[:-1])
    prot_fasta = op.join(wd, 'prodigal','{}_proteins.fasta'.format(sag))
    old_tbl = "/mnt/scgc/simon/simonsproject/jb_vs_test/{plate}/{sag}/summary/{sag}_contigs-summary.csv.gz".format(sag=sag, plate=plate)
    if op.exists(old_tbl) == False:
        continue
    out_tbl = phage_contig_table(cmap, prot_fasta, phage_hits, outfile=None)
    old = pd.read_csv(old_tbl).rename(columns={'Unnamed: 0':'contig'})
    recruit = pd.read_csv(op.join(wd, "diamond","pergenome",'{sag}_proteins_mg_diamond_recruitment_tbl.csv'.format(sag=sag)))
    merged = recruit.merge(out_tbl.reset_index(), on='contig', how='outer')
    result = merged.merge(virus_class(merged), on='contig', how='outer')
    comparison = pd.merge(old[['contig','virus_class','virus_prob']], result[['contig','virus_class','virus_prob']], on='contig')
    comparison.rename(columns = {'virus_class_x':'virus_class_old','virus_prob_x':'virus_prob_old', 'virus_class_y':'virus_class_new','virus_prob_y':'virus_prob_new'}, inplace=True)
    comparison.to_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/{}_old_new_comp.csv".format(sag), index=False)

In [130]:
different = ['AG-910-E05','AG-897-A15','AG-894-C07']

In [151]:
sag = different[2]
plate = "-".join(sag.split("-")[:-1])
prot_fasta = op.join(wd, 'prodigal','{}_proteins.fasta'.format(sag))
old_tbl = "/mnt/scgc/simon/simonsproject/jb_vs_test/{plate}/{sag}/summary/{sag}_contigs-summary.csv.gz".format(sag=sag, plate=plate)
out_tbl = phage_contig_table(cmap, prot_fasta, phage_hits, outfile=None)
old = pd.read_csv(old_tbl).rename(columns={'Unnamed: 0':'contig'})
recruit = pd.read_csv(op.join(wd, "diamond","pergenome",'{sag}_proteins_mg_diamond_recruitment_tbl.csv'.format(sag=sag)))
merged = recruit.merge(out_tbl.reset_index(), on='contig', how='outer')
result = merged.merge(virus_class(merged), on='contig', how='outer')
result.merge(old, on='contig')[['contig','ratio_virus_bacteria_x','ratio_virus_bacteria_y','viral_phage_gene_fraction_x','viral_phage_gene_fraction_y']]

,contig,ratio_virus_bacteria_x,ratio_virus_bacteria_y,viral_phage_gene_fraction_x,viral_phage_gene_fraction_y
0,AG-894-C07_NODE_1,0.144910,0.144283,0.007812,0.007812
1,AG-894-C07_NODE_10,0.091033,0.091213,0.000000,0.000000
2,AG-894-C07_NODE_11,0.274637,0.275056,0.000000,0.041667
3,AG-894-C07_NODE_12,0.071383,0.071383,0.000000,0.000000
4,AG-894-C07_NODE_13,0.175954,0.175954,0.000000,0.000000
5,AG-894-C07_NODE_14,0.491748,0.489716,0.000000,0.000000
6,AG-894-C07_NODE_15,0.075027,0.075012,0.000000,0.000000
7,AG-894-C07_NODE_16,0.145047,0.145047,0.000000,0.000000
8,AG-894-C07_NODE_17,0.082751,0.082781,0.000000,0.000000
9,AG-894-C07_NODE_18,0.126723,0.126723,0.000000,0.000000


Small fraction of outputs from 

In [152]:
result[result['contig'].str.contains('AG-894-C07_NODE_3')]

,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria,p1,p2,total_orfs,viral_phage_gene_fraction,viral2_phage_gene_fraction,virus_class,virus_prob
22,AG-894-C07_NODE_3,10456.0,1607.0,5922080.0,8279226.0,98135.0,0.012869,0.002765,0.214865,1.0,1.0,85,0.011765,0.011765,0.0,1.0
23,AG-894-C07_NODE_30,14.0,716.0,5922080.0,8279226.0,5036.0,0.000336,0.024008,71.499080,0.0,0.0,4,0.000000,0.000000,0.0,1.0
24,AG-894-C07_NODE_31,40.0,31.0,5922080.0,8279226.0,4954.0,0.000975,0.001057,1.083471,0.0,0.0,2,0.000000,0.000000,0.0,1.0
25,AG-894-C07_NODE_32,666.0,111.0,5922080.0,8279226.0,3613.0,0.022265,0.005188,0.233004,0.0,0.0,4,0.000000,0.000000,0.0,1.0
26,AG-894-C07_NODE_33,255.0,10.0,5922080.0,8279226.0,2749.0,0.011204,0.000614,0.054825,0.0,0.0,3,0.000000,0.000000,0.0,1.0
27,AG-894-C07_NODE_34,44.0,2.0,5922080.0,8279226.0,2670.0,0.001990,0.000126,0.063547,0.0,0.0,3,0.000000,0.000000,0.0,1.0
28,AG-894-C07_NODE_35,549.0,93.0,5922080.0,8279226.0,2283.0,0.029045,0.006879,0.236824,0.0,0.0,4,0.000000,0.000000,0.0,1.0


In [143]:
from recruitment_for_vs import construct_recruit_tbl

In [144]:
vir_tsv = "/mnt/scgc/simon/simonsproject/jb_vs_test/AG-894/AG-894-C07/diamond/POV.tsv.gz"
bac_tsv = "/mnt/scgc/simon/simonsproject/jb_vs_test/AG-894/AG-894-C07/diamond/LineP-all.tsv.gz"
read_count_dict = {'vir_reads':5922080, 'bac_reads':8279226}
contig_file = "/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/AG-894/AG-894-C07_contigs.fasta"
result2 = construct_recruit_tbl(vir_tsv, bac_tsv, read_count_dict, contig_file)

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.


In [148]:
result2[result2['contig'].str.contains('AG-894-C07_NODE_3')]

,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria
22,AG-894-C07_NODE_3,10456.0,1607.0,5922080.0,8279226.0,98135.0,0.012869,0.002765,0.214865
23,AG-894-C07_NODE_30,14.0,716.0,5922080.0,8279226.0,5036.0,0.000336,0.024008,71.499080
24,AG-894-C07_NODE_31,40.0,31.0,5922080.0,8279226.0,4954.0,0.000975,0.001057,1.083471
25,AG-894-C07_NODE_32,666.0,111.0,5922080.0,8279226.0,3613.0,0.022265,0.005188,0.233004
26,AG-894-C07_NODE_33,255.0,10.0,5922080.0,8279226.0,2749.0,0.011204,0.000614,0.054825
27,AG-894-C07_NODE_34,44.0,2.0,5922080.0,8279226.0,2670.0,0.001990,0.000126,0.063547
28,AG-894-C07_NODE_35,549.0,93.0,5922080.0,8279226.0,2283.0,0.029045,0.006879,0.236824


,contig,virus_class_x,virus_prob_x,virus_class_y,virus_prob_y
0,AG-891-A17_NODE_1,FALSE,1.000000,0.0,1.000000
1,AG-891-A17_NODE_2,FALSE,1.000000,0.0,1.000000
2,AG-891-A17_NODE_3,FALSE,1.000000,0.0,1.000000
3,AG-891-A17_NODE_4,FALSE,1.000000,0.0,1.000000
4,AG-891-A17_NODE_5,FALSE,1.000000,0.0,1.000000
5,AG-891-A17_NODE_6,FALSE,1.000000,0.0,1.000000
6,AG-891-A17_NODE_7,FALSE,1.000000,0.0,1.000000
7,AG-891-A17_NODE_8,FALSE,1.000000,0.0,1.000000
8,AG-891-A17_NODE_9,TRUE,0.962963,1.0,0.962963
9,AG-891-A17_NODE_10,FALSE,1.000000,0.0,1.000000
